$$
\newcommand{\E}{\text{E}}
\newcommand{\mbf}{\mathbf}
\newcommand{\bs}{\boldsymbol}
\newcommand{\Var}{\text{Var}}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\e}{\frac{1}{\sigma^2_e}}
\newcommand{\f}{\frac{1}{\sigma^2_{\alpha}}}
$$

# Algorithm to invert a positive definite matrix

# Rohan L Fernando

# December 6, 2018


- Suppose $\mbf{A}$ is a positive definite variance-covariance matrix

- Can write:
\begin{align*}
    \begin{bmatrix}
    \mbf{A}_{11} & \mbf{A}_{12} \\
    \mbf{A}_{21} & \mbf{A}_{22}
    \end{bmatrix}
    &= \begin{bmatrix}
    \mbf{A}_{11}         & \mbf{A}_{11}\mbf{L} \\
    \mbf{L}'\mbf{A}_{11} & \mbf{L}'\mbf{A}_{11}\mbf{L} + \mbf{R} 
      \end{bmatrix}\\
\end{align*}

-  $\mbf{L} = \mbf{A}_{11}^{-1}\mbf{A}_{12}$

-  $\mbf{R} = \mbf{A}_{22} - \mbf{L}'\mbf{A}_{11}\mbf{L}$

### Numerical demonstration of above identity

#### First construct PD $\mbf{A}$ matrix:

In [1]:
using Distributions, LinearAlgebra
X = rand(Binomial(),10,5)

10×5 Array{Int64,2}:
 1  0  0  0  1
 1  1  0  0  0
 0  1  1  1  0
 1  0  1  0  0
 0  1  0  0  1
 0  1  1  1  1
 1  0  0  0  0
 1  1  0  0  1
 0  1  0  0  1
 1  1  0  1  0

In [2]:
A = X'X

5×5 Array{Int64,2}:
 6  3  1  1  2
 3  7  2  3  4
 1  2  3  2  1
 1  3  2  3  1
 2  4  1  1  5

In [3]:
s1 = 1:2
s2 = 3:5
A11 = A[s1,s1]
A12 = A[s1,s2]
A21 = A[s2,s1]
A22 = A[s2,s2]

3×3 Array{Int64,2}:
 3  2  1
 2  3  1
 1  1  5

Show that $\mbf{A}_{12}$ can be written as $\mbf{A}_{11}\mbf{L}$

In [4]:
A11i = inv(A11)
L = A11i*A12
round.(A11*L - A12,digits=3)

2×3 Array{Float64,2}:
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0

In fact, any $2\times 1$ vector $\mbf{b}$ can be written as:

$\mbf{A}\mbf{x} = \mbf{b}$

#### Numerical demonstration and adding the difference

In [7]:
b = randn(2)
x = A11i*b
[A11*x b round.(b-A11*x, digits=4)]

2×3 Array{Float64,2}:
 -0.915042  -0.915042  0.0
  1.57086    1.57086   0.0

So, can write 
\begin{align*}
\begin{bmatrix}
    \mbf{A}_{11} & \mbf{A}_{12} 
\end{bmatrix}
&= \mbf{A}_{11}\begin{bmatrix}
                   \mbf{I} & \mbf{L} 
                \end{bmatrix}\\
&= \begin{bmatrix}
   \mbf{A}_{11} & \mbf{A}_{11}\mbf{L} 
                \end{bmatrix},
\end{align*}
which is the first row-block of $\mbf{A}$.

But, because $\mbf{A}_{21} = \mbf{A}'_{12}$, $\mbf{A}_{21}$ can be written as $\mbf{A}_{21} = \mbf{L}'\mbf{A}_{11}$. 

#### Verifying numerically:

In [8]:
round.(L'A11 - A21)

3×2 Array{Float64,2}:
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0

So, the matrix:

$$ 
\mbf{B} = 
\begin{bmatrix}
   \mbf{I} \\
   \mbf{L}'
\end{bmatrix}
\mbf{A}_{11}\begin{bmatrix}
                   \mbf{I} & \mbf{L} 
                \end{bmatrix}
=
\begin{bmatrix}
    \mbf{A}_{11}         & \mbf{A}_{11}\mbf{L} \\
    \mbf{L}'\mbf{A}_{11} & \mbf{L}'\mbf{A}_{11}\mbf{L}
      \end{bmatrix}
$$

differs from $\mbf{A}$ only in that $\mbf{A}_{22}$ may not be equal to $\mbf{L}'\mbf{A}_{11}\mbf{L}$. Can show that $\mbf{B}$ is not positive definite, but $\mbf{A}$ is positive definite. 

In [9]:
Identity(n) = Matrix(I,n,n)
x = rand(5)
B =  [Identity(2)
      L'          ] *A11 *[Identity(2) L]

5×5 Array{Float64,2}:
 6.0  3.0  1.0       1.0       2.0    
 3.0  7.0  2.0       3.0       4.0    
 1.0  2.0  0.575758  0.848485  1.15152
 1.0  3.0  0.848485  1.30303   1.69697
 2.0  4.0  1.15152   1.69697   2.30303

### Note
In matrix B, the second column block is a linear combination of the first column block. Multiplication of the first column block times L is the second column block. Hence already from that, we can see that B cannot be positive definite which is shown in addition with finding a quadratic form $y^TBy$ that is $0$, even for $y \ne 0$. 

In [12]:
y = [L[:,1]
     -1
      0
      0
    ]

5-element Array{Float64,1}:
  0.030303030303030276
  0.2727272727272727  
 -1.0                 
  0.0                 
  0.0                 

In [13]:
B*y

5-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [14]:
y = [L[:,2]
       0
      -1
       0
    ]
B*y

5-element Array{Float64,1}:
 -1.1102230246251565e-16
 -1.6653345369377348e-16
 -4.163336342344337e-17 
  1.8735013540549517e-16
  0.0                   

In [15]:
y'B*y

-2.561082659078486e-16

Can write $\mbf{A}$ as 

\begin{align*}
   \begin{bmatrix}
    \mbf{A}_{11} & \mbf{A}_{12} \\
    \mbf{A}_{21} & \mbf{A}_{22}
    \end{bmatrix}
    &= \begin{bmatrix}
   \mbf{I} \\
   \mbf{L}'
\end{bmatrix}
\mbf{A}_{11}\begin{bmatrix}
                \mbf{I} & \mbf{L} 
             \end{bmatrix}
+
\begin{bmatrix}
   \mbf{0} \\
   \mbf{I}
\end{bmatrix}
\mbf{R}\begin{bmatrix}
              \mbf{0} & \mbf{I} 
        \end{bmatrix}\\
    &= 
    \begin{bmatrix}
     \mbf{I} & \mbf{0} \\
     \mbf{L}' & \mbf{I}
    \end{bmatrix}
    \begin{bmatrix}
     \mbf{A}_{11} & \mbf{0} \\
     \mbf{0}      & \mbf{R}
    \end{bmatrix}
        \begin{bmatrix}
     \mbf{I} & \mbf{L} \\
     \mbf{0} & \mbf{I}
    \end{bmatrix}\\
    &= \mbf{PVP}'
\end{align*}

So, 
$$
\mbf{A}^{-1} = (\mbf{P}')^{-1}\mbf{V}^{-1}\mbf{P}^{-1}
$$

## Inverse of $\mbf{A}$
\begin{align*}
   \begin{bmatrix}
    \mbf{A}^{11} & \mbf{A}^{12} \\
    \mbf{A}^{21} & \mbf{A}^{22}
    \end{bmatrix}
    &= (\mbf{P}')^{-1}\mbf{V}^{-1}\mbf{P}^{-1}\\
    &= 
    \begin{bmatrix}
     \mbf{I} & -\mbf{L} \\
     \mbf{0} & \mbf{I}
    \end{bmatrix}
    \begin{bmatrix}
     \mbf{A}_{11}^{-1} & \mbf{0} \\
     \mbf{0}      & \mbf{R}^{-1}
    \end{bmatrix}
        \begin{bmatrix}
     \mbf{I} & \mbf{0} \\
     -\mbf{L}' & \mbf{I}
    \end{bmatrix} \\
    &= 
    \begin{bmatrix}
     \mbf{I}\\
     \mbf{0}
    \end{bmatrix}
     \mbf{A}_{11}^{-1}
    \begin{bmatrix}
     \mbf{I} & \mbf{0}
    \end{bmatrix}
    +
    \begin{bmatrix}
     -\mbf{L}\\
     \mbf{I}
    \end{bmatrix}
    \mbf{R}^{-1}
    \begin{bmatrix}
     -\mbf{L}' & \mbf{I}
    \end{bmatrix}   
    \end{align*}
    

\begin{align*}
&=
   \begin{bmatrix}
    \mbf{A}_{11}^{-1} & \mbf{0} \\
    \mbf{0}      & \mbf{0}
    \end{bmatrix}
+
   \begin{bmatrix}
    \mbf{L}\mbf{R}^{-1}\mbf{L}' & -\mbf{L}\mbf{R}^{-1} \\
    -\mbf{R}^{-1}\mbf{L}'       & \mbf{R}^{-1}
    \end{bmatrix}
\end{align*}    

### Inverse of P
Assume that the matrix $P$ can be written as

\begin{align*}
 P = \left[
    \begin{array}{lr}
     \mbf{I} & \mbf{0} \\
     \mbf{L}' & \mbf{I}
    \end{array}
 \right]    
\end{align*}

Then $P^{-1}$ can be written as

\begin{align*}
 P^{-1} = \left[
    \begin{array}{lr}
     \mbf{I} & \mbf{0} \\
     -\mbf{L}' & \mbf{I}
    \end{array}
 \right]    
\end{align*}


**Proof**:

\begin{align*}
P^{-1}P &= \left[
    \begin{array}{lr}
     \mbf{I} & \mbf{0} \\
     -\mbf{L}' & \mbf{I}
    \end{array}
 \right] 
 \left[
    \begin{array}{lr}
     \mbf{I} & \mbf{0} \\
     \mbf{L}' & \mbf{I}
    \end{array}
 \right]   
 =  \left[
    \begin{array}{lr}
     \mbf{I} & \mbf{0} \\
     \mbf{0} & \mbf{I}
    \end{array}
 \right]   
\end{align*}

## Inverse Function

In [20]:
function invA(A)
    n = size(A,1)
    if n==1
        return 1/A[1,1]
    else
        s1 = 1:n-1
        s2 = n
        A11 = A[s1,s1]
        A11i = invA(A11)
        A12 = A[s1,s2]
        A22 = A[s2,s2]
        L = A11i*A12
        R = A22 - L'A11*L
        Ri = 1/R[1,1]
        Ai = [A11i .+ (Ri.* L*L') -Ri.*L
              -Ri.*L'             Ri.*1]
        return Ai
   end
end

invA (generic function with 1 method)

## Numerical Example

In [21]:
X = [1 2 3
     3 4 5
     6 7 9]
A = X'X

3×3 Array{Int64,2}:
 46  56   72
 56  69   89
 72  89  115

In [22]:
# Julia inverse function
inv(A)

3×3 Array{Float64,2}:
  3.5   -8.0    4.0
 -8.0   26.5  -15.5
  4.0  -15.5    9.5

In [23]:
# Our inverse function
invA(A)

3×3 Array{Float64,2}:
  3.5   -8.0    4.0
 -8.0   26.5  -15.5
  4.0  -15.5    9.5

In [24]:
### # compare results of both functions
round.(inv(A) - invA(A))

3×3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0

## Additive Relationship Matrix: $\mbf{A}$

- $a_{ij}$ is two times the kinship probability ($F_{ij}$) that randomly sampled alleles from $i$ and $j$ are IBD

- Homologous alleles are IBD if they can be traced to the same ancestral allele in a founder

- Thus, two different alleles in founders are not IBD

- Homologous alleles are randomly inherited one from each of the parents $m_i$ and and $f_i$ of $i$

- If $j$ is not a descendant of $i$, the genetic relationship of $j$ with $i$ is entirely through the parents of $j$

Thus,
$$
F_{ij} = \frac{1}{2}(F_{jm_i} + F_{jf_i})
$$

and 

$$
F_{ii} = \frac{1}{2}(1 + F_{m_if_i})
$$

Similarly,
$$
a_{ij} = \frac{1}{2}(a_{jm_i} + a_{jf_i})
$$

and 

$$
a_{ii} = \frac{1}{2}(2 + a_{m_if_i})
$$

## Tabular Method to Compute $\mbf{A}$

- Number individuals such that parents preced offspring

- For founders, enter 1 on diagonal and 0 on off-diagonals

- For non-founder $i$ calculate row elements 1 to $i-1$ as the average of the parental row elements

- Set diagonal element $i$ to $\frac{1}{2}(1 + a_{m_if_i})$

- Fill columns by symmetry.

In matrix notation:

$$
\mbf{A}_i = 
\begin{bmatrix}
    \mbf{A}_{i-1}           & \mbf{A}_{i-1}\mbf{q}_i \\
    \mbf{q}'_i\mbf{A}_{i-1} & \frac{1}{2}(2 + a_{m_if_i})
      \end{bmatrix}
$$

where $mbf{q}_i$ has only two non-zero elements that are equal to 0.5 at positions corresponding to $m_i$ and $f_i$.

So,
$$
\mbf{A}_i^{-1}=
   \begin{bmatrix}
    \mbf{A}_{i-1}^{-1} & \mbf{0} \\
    \mbf{0}      & \mbf{0}
    \end{bmatrix}
+
    \begin{bmatrix}
     -\mbf{q}_i\\
          1
    \end{bmatrix}
    \mbf{a}^{ii}
    \begin{bmatrix}
     -\mbf{q}'_i & 1
    \end{bmatrix}
$$  

where $a^{ii} = (a_{ii} - \mbf{q}'_i\mbf{A}_{i-1}\mbf{q}_i)^{-1}$

## Algorithm

- Set $\mbf{A}^{-1} = \mbf{0}$

- Compute $a^{ii}$ for all animals

- For each animal add to $\mbf{A}^{-1}$

   - $a^{ii}$ to (i,i)
   - -1/2$a^{ii}$ to $(i,f_i)$, $(f_i,i)$,$(i,m_i)$,$(m_i,i)$
   - 1/4$a^{ii}$ to $(f_i,f_i)$, $(f_i,m_i)$,$(m_i,f_i)$,$(m_i,m_i)$

## Numerical Example

Pedigree:
$$
\begin{matrix}
i & f & m\\
1 & 0 & 0\\
2 & 0 & 0\\
3 & 1 & 2\\
4 & 1 & 2\\
5 & 3 & 4
\end{matrix}
$$

Because the first two individuals are founders, their relationship matrix will be $A2$ which is a $2\times 2$ identity matrix.

In [25]:
A2 = [1 0
      0 1]

2×2 Array{Int64,2}:
 1  0
 0  1

Now we are adding the row and column and the diagonal element for animal 3. Note that animal 3 has parents 1 and 2, hence $q$ is a two element vector with each element is $.5$ and `amf` is the relationship between the parents.

In [57]:
q   = [0.5, 0.5]

2-element Array{Float64,1}:
 0.5
 0.5

In [58]:
A

5×5 Array{Float64,2}:
 1.0  0.0  0.5   0.5   0.5 
 0.0  1.0  0.5   0.5   0.5 
 0.5  0.5  1.0   0.5   0.75
 0.5  0.5  0.5   1.0   0.75
 0.5  0.5  0.75  0.75  1.25

In [26]:
q   = [0.5, 0.5]
amf = A2[1,2]
A3 = [A2    A2*q
      q'A2  1 + 0.5amf   ]

3×3 Array{Float64,2}:
 1.0  0.0  0.5
 0.0  1.0  0.5
 0.5  0.5  1.0

In [27]:
q   = [0.5, 0.5, 0]
amf = A3[1,2]
A4 = [A3    A3*q
      q'A3  1 + 0.5amf   ]

4×4 Array{Float64,2}:
 1.0  0.0  0.5  0.5
 0.0  1.0  0.5  0.5
 0.5  0.5  1.0  0.5
 0.5  0.5  0.5  1.0

In [28]:
q   = [0, 0, 0.5, 0.5]
amf = A4[3,4]
A5 = [A4    A4*q
      q'A4  1 + 0.5amf   ]
A = A5

5×5 Array{Float64,2}:
 1.0  0.0  0.5   0.5   0.5 
 0.0  1.0  0.5   0.5   0.5 
 0.5  0.5  1.0   0.5   0.75
 0.5  0.5  0.5   1.0   0.75
 0.5  0.5  0.75  0.75  1.25

## A Inverse

In [29]:
using LinearAlgebra
A2i = Matrix(1.0I, 2,2)

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [30]:
q = [0.5, 0.5]
a33i = 1/(A[3,3] - q'A2*q)
A3i  = [A2i zero(q)
        zero(q')   0] + [-q; 1]*[-q' 1].*a33i

3×3 Array{Float64,2}:
  1.5   0.5  -1.0
  0.5   1.5  -1.0
 -1.0  -1.0   2.0

In [41]:
### # what is zero(q)
q
zero(q) # creates a matrix with same dimension of q with all elements = 0


2-element Array{Float64,1}:
 0.0
 0.0

In [42]:
inv(A[1:3,1:3])

3×3 Array{Float64,2}:
  1.5   0.5  -1.0
  0.5   1.5  -1.0
 -1.0  -1.0   2.0

In [43]:
q    = [0.5, 0.5, 0]
a44i = 1/(A[4,4] - q'A3*q)
A4i  = [A3i zero(q)
        zero(q')   0] + [-q; 1]*[-q' 1].*a44i

4×4 Array{Float64,2}:
  2.0   1.0  -1.0  -1.0
  1.0   2.0  -1.0  -1.0
 -1.0  -1.0   2.0   0.0
 -1.0  -1.0   0.0   2.0

In [44]:
inv(A[1:4,1:4])

4×4 Array{Float64,2}:
  2.0   1.0  -1.0  -1.0
  1.0   2.0  -1.0  -1.0
 -1.0  -1.0   2.0  -0.0
 -1.0  -1.0   0.0   2.0

In [45]:
q    = [0, 0, 0.5, 0.5]
[-q; 1]*[-q' 1]


5×5 Array{Float64,2}:
  0.0   0.0   0.0    0.0   -0.0
  0.0   0.0   0.0    0.0   -0.0
  0.0   0.0   0.25   0.25  -0.5
  0.0   0.0   0.25   0.25  -0.5
 -0.0  -0.0  -0.5   -0.5    1.0

In [46]:
a55i = 1/(A[5,5] - q'A4*q)
A5i  = [A4i zero(q)
        zero(q')   0] + [-q; 1]*[-q' 1].*a55i

5×5 Array{Float64,2}:
  2.0   1.0  -1.0  -1.0   0.0
  1.0   2.0  -1.0  -1.0   0.0
 -1.0  -1.0   2.5   0.5  -1.0
 -1.0  -1.0   0.5   2.5  -1.0
  0.0   0.0  -1.0  -1.0   2.0

In [47]:
inv(A)

5×5 Array{Float64,2}:
  2.0   1.0  -1.0  -1.0  -0.0
  1.0   2.0  -1.0  -1.0  -0.0
 -1.0  -1.0   2.5   0.5  -1.0
 -1.0  -1.0   0.5   2.5  -1.0
  0.0   0.0  -1.0  -1.0   2.0

In [48]:
# conversion of nb to slides
#; jupyter nbconvert --to slides AInverse.ipynb

[NbConvertApp] Converting notebook AInverse.ipynb to slides
[NbConvertApp] Writing 306689 bytes to AInverse.slides.html


In [56]:
### # similarly with matrices of all $1$
q = [1 2;3 4]
#ones(q)
#one(q)
one.(q)


2×2 Array{Int64,2}:
 1  1
 1  1

## Exercise: Convert Recursive Function For Matrix Inversion Into Iterative Function

In [111]:
function recAinv(A)
    ### # get the number of rows of A
    nrowA = size(A, 1)
    ### # do the first element
    AinvResult = 1/A[1,1]
    ### # loop over rows in A starting in row 2
    for i=2:nrowA
        
        ### # for the following rows
        q = A[1:(i-1),i] 
        aii = 1/(A[i,i] - q'A[1:(i-1),1:(i-1)]*q)
        AinvResult = [AinvResult zero(q)
                      zero(q')        0] + [-q; 1]*[-q' 1].*aii
    end
    return AinvResult
end



recAinv (generic function with 1 method)

Testing the function with the above given matrices leads to

In [112]:
A

5×5 Array{Float64,2}:
 1.0  0.0  0.5   0.5   0.5 
 0.0  1.0  0.5   0.5   0.5 
 0.5  0.5  1.0   0.5   0.75
 0.5  0.5  0.5   1.0   0.75
 0.5  0.5  0.75  0.75  1.25

In [113]:
AinvRec = recAinv(A)

5×5 Array{Float64,2}:
  0.397436  -0.602564  -2.15385    1.84615    0.205128
 -0.602564   0.397436  -2.15385    1.84615    0.205128
 -2.15385   -2.15385    0.769231   1.76923    0.307692
  1.84615    1.84615    1.76923   -4.23077    0.307692
  0.205128   0.205128   0.307692   0.307692  -0.410256

In [92]:
inv(A)

5×5 Array{Float64,2}:
  2.0   1.0  -1.0  -1.0  -0.0
  1.0   2.0  -1.0  -1.0  -0.0
 -1.0  -1.0   2.5   0.5  -1.0
 -1.0  -1.0   0.5   2.5  -1.0
  0.0   0.0  -1.0  -1.0   2.0

## Unrolling to Debug
Frist get a smaller matrix and unroll the function

Start with 2x2


In [114]:
testA2 = copy(A[1:2,1:2])

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [115]:
AinvResult = zero(testA2)

2×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0

In [116]:
nrowA = size(testA2, 1)

2

In [117]:
AinvResult[1,1] = 1/testA2[1,1]

1.0

In [118]:
i=2

2

In [119]:
q = testA2[1:(i-1),i] 

1-element Array{Float64,1}:
 0.0

In [120]:
aii = 1/(testA2[i,i] - q'testA2[(i-1),(i-1)]*q)

1.0

In [121]:
AinvResult[(1:i),(1:i)]


2×2 Array{Float64,2}:
 1.0  0.0
 0.0  0.0

In [122]:
[-q; 1]*[-q' 1].*aii

2×2 Array{Float64,2}:
  0.0  -0.0
 -0.0   1.0

In [123]:
AinvResult[(1:i),(1:i)] = AinvResult[(1:i),(1:i)] + [-q; 1]*[-q' 1].*aii

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

Continue with 3x3

In [124]:
testA3 = copy(A[1:3,1:3])

3×3 Array{Float64,2}:
 1.0  0.0  0.5
 0.0  1.0  0.5
 0.5  0.5  1.0

In [125]:
AinvResult = zero(testA3)

3×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [126]:
nrowA = size(testA3, 1)

3

In [127]:
AinvResult[1,1] = 1/testA3[1,1]

1.0

In [128]:
i=2

2

In [129]:
q = A[1:(i-1),i]

1-element Array{Float64,1}:
 0.0

In [131]:
aii = 1/(A[i,i] - q'A[1:(i-1),1:(i-1)]*q)

1.0

In [132]:
AinvResult[(1:i),(1:i)] = AinvResult[(1:i),(1:i)] + [-q; 1]*[-q' 1].*aii

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [133]:
i=3

3

In [134]:
q = A[1:(i-1),i]

2-element Array{Float64,1}:
 0.5
 0.5

In [135]:
aii = 1/(A[i,i] - q'A[1:(i-1),1:(i-1)]*q)

2.0

In [136]:
AinvResult[(1:i),(1:i)] = AinvResult[(1:i),(1:i)] + [-q; 1]*[-q' 1].*aii

3×3 Array{Float64,2}:
  1.5   0.5  -1.0
  0.5   1.5  -1.0
 -1.0  -1.0   2.0

In [137]:
AinvResult * testA3

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

Continue with 4x4

In [138]:
testA4 = A[1:4,1:4]

4×4 Array{Float64,2}:
 1.0  0.0  0.5  0.5
 0.0  1.0  0.5  0.5
 0.5  0.5  1.0  0.5
 0.5  0.5  0.5  1.0

In [139]:
nrowA = size(testA4, 1)

4

In [140]:
AinvResult[1,1] = 1/A[1,1]

1.0

In [141]:
i=2

2

In [142]:
q = testA4[1:(i-1),i]

1-element Array{Float64,1}:
 0.0

In [143]:
aii = 1/(testA4[i,i] - q'testA4[1:(i-1),1:(i-1)]*q)
        AinvResult[(1:i),(1:i)] = AinvResult[(1:i),(1:i)] + [-q; 1]*[-q' 1].*aii

2×2 Array{Float64,2}:
 1.0  0.5
 0.5  2.5

In [144]:
i=3

3

In [145]:
q = testA4[1:(i-1),i] 

2-element Array{Float64,1}:
 0.5
 0.5

In [146]:
aii = 1/(A[i,i] - q'A[1:(i-1),1:(i-1)]*q)

2.0

In [147]:
AinvResult[(1:i),(1:i)] = AinvResult[(1:i),(1:i)] + [-q; 1]*[-q' 1].*aii

3×3 Array{Float64,2}:
  1.5   1.0  -2.0
  1.0   3.0  -2.0
 -2.0  -2.0   4.0

In [148]:
i=4

4

In [149]:
q = testA4[1:(i-1),i] 

3-element Array{Float64,1}:
 0.5
 0.5
 0.5

In [150]:
aii = 1/(A[i,i] - q'A[1:(i-1),1:(i-1)]*q)

-4.0

In [152]:
AinvResult = [AinvResult zero(q)
                      zero(q')        0] + [-q; 1]*[-q' 1].*aii

4×4 Array{Float64,2}:
  0.5   0.0  -3.0   2.0
  0.0   2.0  -3.0   2.0
 -3.0  -3.0   3.0   2.0
  2.0   2.0   2.0  -4.0

In [109]:
testA4 * AinvResult

4×4 Array{Float64,2}:
  0.5  -0.5  -0.5   1.0
 -0.5   0.5  -0.5   1.0
 -1.0  -1.0   0.0   2.0
  1.0   1.0   0.5  -1.0